In [1]:
import tabula
from tabula import read_pdf
import pandas as pd

In [2]:
#Number of pages in the pdf document
df = tabula.read_pdf("Postcode.pdf")

print (len(df))

33


In [3]:
# Read the different PDF pages into a list of dataframes 
multi_tables = tabula.read_pdf("Postcode.pdf", pages="all", multiple_tables=True, lattice=True)
print (len(multi_tables)) #Number of generated dataframes

1789


In [20]:
for table in multi_tables[:10]:
    print (table.head(10))

           0         1                 2         3              4  \
0     REGION  POSTCODE              WARD  POSTCODE  OLD\rPOSTCODE   
1  ILALA CBD        11          KIVUKONI     11101            NaN   
2        NaN       NaN               NaN       NaN            NaN   
3        NaN       NaN  UPANGA MASHARIKI     11102            NaN   
4        NaN       NaN               NaN       NaN            NaN   
5        NaN       NaN  UPANGA MAGHARIBI     11103            NaN   
6        NaN       NaN               NaN       NaN            NaN   
7        NaN       NaN               NaN       NaN            NaN   
8        NaN       NaN            KISUTU     11104            NaN   
9        NaN       NaN               NaN       NaN            NaN   

              5          6  
0  MTAA/VILLAGE  KITONGOJI  
1      Kivukoni        NaN  
2      Sea View        NaN  
3       Kitonga        NaN  
4      Kibasila        NaN  
5          Fire        NaN  
6      Charambe        NaN  
7       

In [21]:
print (multi_tables[0].columns.values)
# Sample of the tables' headers -- this will need to be changed later. 

[0 1 2 3 4 5 6]


##### Attempt to combine the different pages into regional tables for future combination 

In [68]:
def combine_tables(multi_tables):
    # Combine the dataframes belonging to the same regions
    # into one dataframe per region. 
    df_list = []
    
    for table in multi_tables:
        if table.loc[0,0] == 'REGION':
            try:
                df_list.append(temp_df)
            except UnboundLocalError:
                pass
            
            temp_df = table
        else:
            temp_df = pd.concat([temp_df, table])
    return df_list
            

In [69]:
regions = combine_tables(multi_tables)

In [70]:
for region in regions:
    print (region.iloc[1,0])
    #List of all regions with dataframes
    

ILALA CBD
TANGA
ARUSHA
KILIMANJARO
MANYARA
GEITA
MARA
MWANZA
KAGERA
SHINYANGA
SIMIYU
DODOMA
SINGIDA
TABORA
KIGOMA
KATAVI
IRINGA
MBEYA
RUKWA
RUVUMA
NJOMBE
PWANI
MTWARA
LINDI


In [71]:
def create_columns(df):
    #extract the first row of the table and replace
    #the column headers with the first row.
    columns = df.columns.values
    #new_cols = df.iloc[0, columns].values
    new_cols = df.iloc[0]
    print ('Columns: {}, New columns: {}'.format(columns, new_cols))
    
    rename_dict = {}
    '''
    POSTCODE - the first two digits of the postcode
    POSTCODE2 = the first 3 digits of the postcode
    POSTCODE3 = the 6 digit postcode
    '''
    for column in columns:
        if column == 3:
            rename_dict[column] = 'POSTCODE2'
        elif column == 5:
            rename_dict[column] = 'POSTCODE3'
        else:
            rename_dict[column] = new_cols[column]
    
    df.rename(columns=rename_dict, inplace=True)
    return df

def create_dsm_columns(df):
    #Because the DSM table format is different
    columns = df.columns.values
    new_cols = df.iloc[0]
    
    rename_dict = {}
    '''
    POSTCODE - the first two digits of the postcode
    POSTCODE3 = the 6 digit postcode 
    '''
    for column in columns:
        if column == 3:
            rename_dict[column] = 'POSTCODE3'
        else:
            rename_dict[column] = new_cols[column]
    
    df.rename(columns=rename_dict, inplace=True)
    return df
    
    
    
def clean_table(df):
    #perform forward fill for each column in the tables
    
    for column in df:
        df[column] = df[column].fillna(method='ffill')
    return df

In [72]:
new_regions = []

temp = create_dsm_columns(regions[0])
new_regions.append(temp)


for region in regions[1:]:
    temp = create_columns(region)
    new_regions.append(temp)
    

Columns: [0 1 2 3 4 5 6 7 8], New columns: 0           REGION
1         POSTCODE
2         DISTRICT
3         POSTCODE
4             WARD
5         POSTCODE
6    OLD\rPOSTCODE
7     MTAA/VILLAGE
8        KITONGOJI
Name: 0, dtype: object
Columns: [0 1 2 3 4 5 6 7], New columns: 0          REGION
1        POSTCODE
2        DISTRICT
3        POSTCODE
4            WARD
5        POSTCODE
6    MTAA/VILLAGE
7       KITONGOJI
Name: 0, dtype: object
Columns: [0 1 2 3 4 5 6 7 8], New columns: 0           REGION
1         POSTCODE
2         DISTRICT
3         POSTCODE
4             WARD
5         POSTCODE
6    OLD\rPOSTCODE
7     MTAA/VILLAGE
8        KITONGOJI
Name: 0, dtype: object
Columns: [0 1 2 3 4 5 6 7], New columns: 0          REGION
1        POSTCODE
2        DISTRICT
3        POSTCODE
4            WARD
5        POSTCODE
6    MTAA/VILLAGE
7       KITONGOJI
Name: 0, dtype: object
Columns: [0 1 2 3 4 5 6 7], New columns: 0          REGION
1        POSTCODE
2        DISTRICT
3        POSTCO

In [73]:
new_regions[0].head(5)

,REGION,POSTCODE,WARD,POSTCODE3,OLD POSTCODE,MTAA/VILLAGE,KITONGOJI
0,REGION,POSTCODE,WARD,POSTCODE,OLD\rPOSTCODE,MTAA/VILLAGE,KITONGOJI
1,ILALA CBD,11,KIVUKONI,11101,NaN,Kivukoni,NaN
2,NaN,NaN,NaN,NaN,NaN,Sea View,NaN
3,NaN,NaN,UPANGA MASHARIKI,11102,NaN,Kitonga,NaN
4,NaN,NaN,NaN,NaN,NaN,Kibasila,NaN


In [74]:
final_regions = []
for region in new_regions:
    temp = clean_table(region)
    final_regions.append(temp)

In [75]:
final_regions[0].tail(100)

,REGION,POSTCODE,WARD,POSTCODE3,OLD POSTCODE,MTAA/VILLAGE,KITONGOJI
26,UBUNGO,16,Kibamba,16110,14127,Kiluvya,KITONGOJI
27,UBUNGO,16,Kwembe,16111,14133,Kwembe,KITONGOJI
28,UBUNGO,16,Kwembe,16111,14133,Mpakani,KITONGOJI
29,UBUNGO,16,Kwembe,16111,14133,King'azi,KITONGOJI
30,UBUNGO,16,Kwembe,16111,14133,Luguruni,KITONGOJI
31,UBUNGO,16,Kwembe,16111,14133,Kisopwa,KITONGOJI
32,UBUNGO,16,Kwembe,16111,14133,Njeteni,KITONGOJI
33,UBUNGO,16,Kwembe,16111,14133,king'azi B,KITONGOJI
0,UBUNGO,16,Kwembe,16111,14133,Mjimpya,KITONGOJI
1,UBUNGO,16,Kwembe,16111,14133,Msakuzi,KITONGOJI


#### Attempting to combine all the regions into one table 

In [76]:
for region in final_regions:
    print (region.columns.values)

['REGION' 'POSTCODE' 'WARD' 'POSTCODE3' 'OLD\rPOSTCODE' 'MTAA/VILLAGE'
 'KITONGOJI']
['REGION' 'POSTCODE' 'DISTRICT' 'POSTCODE2' 'WARD' 'POSTCODE3'
 'OLD\rPOSTCODE' 'MTAA/VILLAGE' 'KITONGOJI']
['REGION' 'POSTCODE' 'DISTRICT' 'POSTCODE2' 'WARD' 'POSTCODE3'
 'MTAA/VILLAGE' 'KITONGOJI']
['REGION' 'POSTCODE' 'DISTRICT' 'POSTCODE2' 'WARD' 'POSTCODE3'
 'OLD\rPOSTCODE' 'MTAA/VILLAGE' 'KITONGOJI']
['REGION' 'POSTCODE' 'DISTRICT' 'POSTCODE2' 'WARD' 'POSTCODE3'
 'MTAA/VILLAGE' 'KITONGOJI']
['REGION' 'POSTCODE' 'DISTRICT' 'POSTCODE2' 'WARD' 'POSTCODE3'
 'MTAA/VILLAGE' 'KITONGOJI']
['REGION' 'POSTCODE' 'DISTRICT' 'POSTCODE2' 'WARD' 'POSTCODE3'
 'OLD\rPOSTCODE' 'MTAA/VILLAGE' 'KITONGOJI']
['REGION' 'POSTCODE' 'DISTRICT' 'POSTCODE2' 'WARD' 'POSTCODE3'
 'OLD\rPOSTCODE' 'MTAA/VILLAGE' 'KITONGOJI']
['REGION' 'POSTCODE' 'DISTRICT' 'POSTCODE2' 'WARD' 'POSTCODE3'
 'MTAA/VILLAGE' 'KITONGOJI']
['REGION' 'POSTCODE' 'DISTRICT' 'POSTCODE2' 'WARD' 'POSTCODE3'
 'OLD\rPOSTCODE' 'MTAA/VILLAGE' 'KITONGOJI']
['REGIO

In [77]:
# Add the 'DISTRICT' and 'MTAA/VILLAGE' columns in the first region table.

final_regions[0]['DISTRICT'] = None
final_regions[0]['MTAA/VILLAGE'] = None

In [78]:
trimmed_regions = []
for region in final_regions:
    temp_df = region[['REGION', 'DISTRICT', 'WARD', 'MTAA/VILLAGE', 'KITONGOJI', 'POSTCODE3']]
    trimmed_regions.append(temp_df)
    
trimmed_regions[0].head(10)

,REGION,DISTRICT,WARD,MTAA/VILLAGE,KITONGOJI,POSTCODE3
0,REGION,None,WARD,None,KITONGOJI,POSTCODE
1,ILALA CBD,None,KIVUKONI,None,KITONGOJI,11101
2,ILALA CBD,None,KIVUKONI,None,KITONGOJI,11101
3,ILALA CBD,None,UPANGA MASHARIKI,None,KITONGOJI,11102
4,ILALA CBD,None,UPANGA MASHARIKI,None,KITONGOJI,11102
5,ILALA CBD,None,UPANGA MAGHARIBI,None,KITONGOJI,11103
6,ILALA CBD,None,UPANGA MAGHARIBI,None,KITONGOJI,11103
7,ILALA CBD,None,UPANGA MAGHARIBI,None,KITONGOJI,11103
8,ILALA CBD,None,KISUTU,None,KITONGOJI,11104
9,ILALA CBD,None,KISUTU,None,KITONGOJI,11104


In [80]:
final = trimmed_regions[0]

for region in trimmed_regions[1:]:
    final = pd.concat([final, region])
    
    
final.head(10) 

,REGION,DISTRICT,WARD,MTAA/VILLAGE,KITONGOJI,POSTCODE3
0,REGION,None,WARD,None,KITONGOJI,POSTCODE
1,ILALA CBD,None,KIVUKONI,None,KITONGOJI,11101
2,ILALA CBD,None,KIVUKONI,None,KITONGOJI,11101
3,ILALA CBD,None,UPANGA MASHARIKI,None,KITONGOJI,11102
4,ILALA CBD,None,UPANGA MASHARIKI,None,KITONGOJI,11102
5,ILALA CBD,None,UPANGA MAGHARIBI,None,KITONGOJI,11103
6,ILALA CBD,None,UPANGA MAGHARIBI,None,KITONGOJI,11103
7,ILALA CBD,None,UPANGA MAGHARIBI,None,KITONGOJI,11103
8,ILALA CBD,None,KISUTU,None,KITONGOJI,11104
9,ILALA CBD,None,KISUTU,None,KITONGOJI,11104


In [86]:
print (len(final))
final = final[final.REGION != 'REGION']
final.rename(columns={'POSTCODE3': 'POSTCODE'}, inplace=True)

final.tail(10)

65217


,REGION,DISTRICT,WARD,MTAA/VILLAGE,KITONGOJI,POSTCODE
26,LINDI,Ruangwa,Mbwemkuru (Machang'anja),Machang'anja,Mtakuja,65622
27,LINDI,Ruangwa,Mbwemkuru (Machang'anja),Machang'anja,Mbuyuni,65622
28,LINDI,Ruangwa,Mbwemkuru (Machang'anja),Machang'anja,Kajiungeni,65622
29,LINDI,Ruangwa,Mbwemkuru (Machang'anja),Machang'anja,Majengo,65622
30,LINDI,Ruangwa,Mbwemkuru (Machang'anja),Machang'anja,Njenga,65622
31,LINDI,Ruangwa,Mbwemkuru (Machang'anja),Chikwale,Chikwale,65622
32,LINDI,Ruangwa,Mbwemkuru (Machang'anja),Chikwale,Namaputu,65622
33,LINDI,Ruangwa,Mbwemkuru (Machang'anja),Chiundu,Chiundu,65622
34,LINDI,Ruangwa,Mbwemkuru (Machang'anja),Chiundu,Nanjenjema,65622
35,LINDI,Ruangwa,Mbwemkuru (Machang'anja),Chiundu,Nkachile,65622
